# Albeno effekt
---


#### Kode 1


Denne koden henter ut albedo effekten i alpene fra 2004 til 2024 hvor feil marginen er lav fra data_m-lavfeilmargin. Deretter tar den gjennomsnitt og er fremvist med en linje graf med slider for å se ulike positive og negative økninger mellom år. Trendlinjen gir brukeren en lett oversiktielig måte å forstå dataen


In [ ]:
import pandas as pd
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display

# Henter inn referanseår og definer faste snøpunkter
referanse_2004 = pd.read_csv("../data/csv_albedo_effekt/Albedo effekt 2004.csv")
feilmargin    = pd.read_csv("../data/csv_albedo_effekt/data_m-lavfeilmargin.csv")

felles_2004 = pd.merge(referanse_2004, feilmargin, on=["lat","lon"], how="inner")
ref_punkter = felles_2004[felles_2004["AL-BB-DH"] > 0.3][["lat","lon"]]

# Regner ut Årlig gjennomsnittlig albedo
årstall, gjennomsnitt = [], []
for fil in sorted(glob.glob("../data/csv_albedo_effekt/Albedo*.csv")):
    df = pd.read_csv(fil)
    df["lat"] = df["lat"].round(6); df["lon"] = df["lon"].round(6)
    felles = pd.merge(ref_punkter, df, on=["lat","lon"], how="inner")
    årstall.append(int(''.join(filter(str.isdigit, os.path.basename(fil)))))
    gjennomsnitt.append(felles["AL-BB-DH"].mean() if "AL-BB-DH" in felles else np.nan)

stats = pd.DataFrame({"År": årstall, "Gjennomsnittlig albedo": gjennomsnitt})


# Interaktiv funksjon for plotting
def plot_interactive(year_range, show_trend):
    min_year, max_year = year_range
    df = stats[(stats["År"] >= min_year) & (stats["År"] <= max_year)]
    plt.figure(figsize=(10, 5))
    sns.lineplot(data=df, x="År", y="Gjennomsnittlig albedo", marker="o", label="Årlig gj.snitt")
    if show_trend and len(df) > 1:
        sns.regplot(data=df, x="År", y="Gjennomsnittlig albedo", scatter=False,
                    label="Trendlinje", ci=None, line_kws={"linestyle":"--"})
            # Prosentvis endring per år
        koeff = np.polyfit(df["År"], df["Gjennomsnittlig albedo"], 1)
        stigning = koeff[0]
        snitt_albedo = df["Gjennomsnittlig albedo"].mean()
        prosent_endring = (stigning / snitt_albedo) * 100

        retning = "økning" if prosent_endring > 0 else "nedgang"
        print(f"📉 Trend: {prosent_endring:.2f}% {retning} i albedo per år ({stigning:.5f} absolutt endring)")

    plt.title(f"Albedo-utvikling {min_year}–{max_year}")
    plt.xlabel("År"); plt.ylabel("Albedo")
    plt.grid(True, linestyle="--", alpha=0.5)
    plt.legend()
    plt.show()

# Lager interaktive widgeter
year_slider = widgets.IntRangeSlider(
    value=[stats["År"].min(), stats["År"].max()],
    min=stats["År"].min(), max=stats["År"].max(),
    step=1, description="År-range:"
)
trend_toggle = widgets.Checkbox(value=True, description="Vis trendlinje")

interactive_plot = widgets.interactive(plot_interactive,
                                       year_range=year_slider,
                                       show_trend=trend_toggle)

display(interactive_plot)


interactive(children=(IntRangeSlider(value=(2004, 2024), description='År-range:', max=2024, min=2004), Checkbo…